In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [37]:
query0 = """
SELECT *
FROM 
`bigquery-public-data.new_york.311_service_requests` 
WHERE complaint_type = 'HEAT/HOT WATER'
LIMIT 1

"""
pd.read_gbq(query0, project_id='np-hacks',verbose=True, private_key='auth_key.json', dialect='standard')

Requesting query... ok.
Job ID: job_ZIiK_QVxDtfJ7P4pasmnDO5jlZwm
Query running...
Query done.
Processed: 9.6 GB
Standard price: $0.05 USD

Retrieving results...
Got 1 rows.

Total time taken 1.78 s.
Finished at 2017-09-23 15:49:49.


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,address_type,city,landmark,facility_type,status,due_date,resolution_description,resolution_action_updated_date,community_board,borough,x_coordinate,y_coordinate,park_facility_name,park_borough,school_name,school_number,school_region,school_code,school_phone_number,school_address,school_city,school_state,school_zip,school_not_found,school_or_citywide_complaint,vehicle_type,taxi_company_borough,taxi_pickup_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,garage_lot_name,ferry_direction,ferry_terminal_name,latitude,longitude,location
0,29179916,2014-10-30,2014-10-30,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,11373,83-06 VIETOR AVENUE,VIETOR AVENUE,BROADWAY,JUDGE STREET,,,ADDRESS,Elmhurst,,N/A,Closed,NaT,The Department of Housing Preservation and Dev...,2014-10-30,04 QUEENS,QUEENS,1017046,209871,Unspecified,QUEENS,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,,,,,,,,,,,,,40.742663,-73.881647,"(40.74266320977145, -73.881647025061)"


In [42]:
query = """
SELECT created_date, closed_date, resolution_action_updated_date, descriptor,
    location_type, incident_address, incident_zip, community_board,
    borough, latitude, longitude
FROM 
`bigquery-public-data.new_york.311_service_requests` 
WHERE complaint_type = 'HEAT/HOT WATER'
--LIMIT 1000

"""

In [43]:
df = pd.read_gbq(query, project_id='np-hacks',verbose=True, private_key='auth_key.json', dialect='standard')


Requesting query... ok.
Job ID: job_RPyM4MOBu7M0am4K89DCITeqO6Ov
Query running...
  Elapsed 10.9 s. Waiting...
Query done.
Processed: 2.0 GB
Standard price: $0.01 USD

Retrieving results...
  Got page: 1; 5.0% done. Elapsed 17.9 s.
  Got page: 2; 9.0% done. Elapsed 24.23 s.
  Got page: 3; 14.0% done. Elapsed 30.47 s.
  Got page: 4; 18.0% done. Elapsed 37.06 s.
  Got page: 5; 23.0% done. Elapsed 43.15 s.
  Got page: 6; 27.0% done. Elapsed 49.06 s.
  Got page: 7; 32.0% done. Elapsed 55.1 s.
  Got page: 8; 36.0% done. Elapsed 61.35 s.
  Got page: 9; 41.0% done. Elapsed 67.22 s.
  Got page: 10; 45.0% done. Elapsed 73.49 s.
  Got page: 11; 50.0% done. Elapsed 80.42 s.
  Got page: 12; 54.0% done. Elapsed 86.75 s.
  Got page: 13; 59.0% done. Elapsed 92.08 s.
  Got page: 14; 63.0% done. Elapsed 99.27 s.
  Got page: 15; 68.0% done. Elapsed 105.59 s.
  Got page: 16; 72.0% done. Elapsed 111.86 s.
  Got page: 17; 77.0% done. Elapsed 118.31 s.
  Got page: 18; 81.0% done. Elapsed 124.32 s.
  Got pag

In [44]:
df.head()

,created_date,closed_date,resolution_action_updated_date,descriptor,location_type,incident_address,incident_zip,community_board,borough,latitude,longitude
0,2015-11-28 09:32:18,2015-12-01 10:15:48,2015-12-01 10:15:48,APARTMENT ONLY,RESIDENTIAL BUILDING,20-12 SEAGIRT BOULEVARD,11691,14 QUEENS,QUEENS,40.595172,-73.754764
1,2015-11-23 23:25:28,2015-11-24 09:30:53,2015-11-24 09:30:53,ENTIRE BUILDING,RESIDENTIAL BUILDING,20-12 SEAGIRT BOULEVARD,11691,14 QUEENS,QUEENS,40.595172,-73.754764
2,2015-12-20 00:45:47,2015-12-21 15:55:53,2015-12-21 15:55:53,ENTIRE BUILDING,RESIDENTIAL BUILDING,20-12 SEAGIRT BOULEVARD,11691,14 QUEENS,QUEENS,40.595172,-73.754764
3,2017-01-16 01:32:35,2017-01-24 02:15:27,2017-01-24 02:15:27,ENTIRE BUILDING,RESIDENTIAL BUILDING,20-12 SEAGIRT BOULEVARD,11691,14 QUEENS,QUEENS,40.595172,-73.754764
4,2016-12-04 11:27:22,2016-12-08 10:13:31,2016-12-08 10:13:31,ENTIRE BUILDING,RESIDENTIAL BUILDING,20-12 SEAGIRT BOULEVARD,11691,14 QUEENS,QUEENS,40.595172,-73.754764


In [45]:
df.to_csv("hhw.csv")

In [30]:
# per fiscal year, total calls and mean time to resolution
query2 = """
SELECT AVG(close_days) AS avg_close_days,
    count(*) AS n,
    fiscal_year
FROM
(
SELECT DATETIME_DIFF(DATETIME(closed_date), DATETIME(created_date), DAY) AS close_days, 
    EXTRACT(YEAR FROM DATETIME_ADD(DATETIME(created_date), INTERVAL 6 MONTH)) AS fiscal_year
--     created_date,
--     closed_date
FROM 
`bigquery-public-data.new_york.311_service_requests` 
WHERE complaint_type = 'HEAT/HOT WATER'
) x
GROUP BY fiscal_year
ORDER BY fiscal_year
LIMIT 1000

"""


In [31]:
df2 = pd.read_gbq(query2, project_id='np-hacks',verbose=True, private_key='auth_key.json', dialect='standard')

Requesting query... ok.
Job ID: job_2YzZpEla_xhvH24RVjTMdTdEt50k
Query running...
Query done.
Processed: 513.6 MB
Standard price: $0.00 USD

Retrieving results...
Got 5 rows.

Total time taken 2.03 s.
Finished at 2017-09-23 13:35:13.


In [32]:
df2

,avg_close_days,n,fiscal_year
0,3.337840,34853,2014
1,3.739793,240710,2015
2,3.541129,210362,2016
3,3.034665,210422,2017
4,4.135277,6831,2018


In [33]:
df2.to_csv("hhw_summary.csv")

In [34]:
# by borough
# per fiscal year, total calls and mean time to resolution
query3 = """
SELECT AVG(close_days) AS avg_close_days,
    count(*) AS n,
    borough,
    fiscal_year
FROM
(
SELECT DATETIME_DIFF(DATETIME(closed_date), DATETIME(created_date), DAY) AS close_days, 
    EXTRACT(YEAR FROM DATETIME_ADD(DATETIME(created_date), INTERVAL 6 MONTH)) AS fiscal_year,
    borough
FROM 
`bigquery-public-data.new_york.311_service_requests` 
WHERE complaint_type = 'HEAT/HOT WATER'
) x
GROUP BY borough, fiscal_year
ORDER BY borough, fiscal_year
LIMIT 1000

"""


In [35]:
df3 = pd.read_gbq(query3, project_id='np-hacks',verbose=True, private_key='auth_key.json', dialect='standard')
df3.to_csv("hhw_summary_borough.csv")

Requesting query... ok.
Job ID: job_1P0CLoYgpDek6Dcgf5C7yIx7daS4
Query running...
Query done.
Processed: 662.7 MB
Standard price: $0.00 USD

Retrieving results...
Got 25 rows.

Total time taken 2.08 s.
Finished at 2017-09-23 13:44:07.


In [36]:
df3

,avg_close_days,n,borough,fiscal_year
0,2.892283,12015,BRONX,2014
1,3.259046,79670,BRONX,2015
2,3.106135,68159,BRONX,2016
3,3.357202,67982,BRONX,2017
4,3.205319,1950,BRONX,2018
5,3.379777,10137,BROOKLYN,2014
6,3.960647,72310,BROOKLYN,2015
7,3.931229,62607,BROOKLYN,2016
8,2.445958,65302,BROOKLYN,2017
9,4.508764,2292,BROOKLYN,2018
